## Cataract Prediction


In [1]:
# import Necessary Libraries 
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

In [2]:
# Set the path to your dataset
dataset_dir = "C:/Cognitive Learning/Dataset"

In [3]:
# Get a list of all image files in the dataset directory
all_images = []
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith(".png"):
            all_images.append(os.path.join(root, file))

print(f"Number of images: {len(all_images)}")

Number of images: 1007


In [7]:
# Create labels based on folder names (assuming binary classification)
labels = [1 if "cataract" in img else 0 for img in all_images]

In [8]:
# Create a DataFrame with file paths and labels
df = pd.DataFrame({'filename': all_images, 'label': labels})

In [9]:
# Convert labels to strings
df['label'] = df['label'].astype(str)

In [10]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])


In [11]:
# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

In [12]:
# Count the number of images in each category
normal_count = df[df['label'] == '0'].shape[0]
cataract_count = df[df['label'] == '1'].shape[0]

print(f"Number of Normal Images: {normal_count}")
print(f"Number of Cataract Images: {cataract_count}")

Number of Normal Images: 603
Number of Cataract Images: 404


In [13]:
# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [14]:
# Rescaling for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
 #Load and augment training data
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='label',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 805 validated image filenames belonging to 2 classes.


In [16]:
# Load and rescale test data
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='label',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 202 validated image filenames belonging to 2 classes.


In [17]:
# Increase model complexity with additional layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

for layer in base_model.layers:
    layer.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))



58889256/58889256 [==============================] - 5s 0us/step


In [18]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
# Increase the number of epochs
epochs = 20

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_df) // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=len(test_df) // batch_size
)

Epoch 1/20


25/25 [==============================] - 173s 7s/step - loss: 2.4863 - accuracy: 0.5498 - val_loss: 0.5396 - val_accuracy: 0.7396
Epoch 2/20
25/25 [==============================] - 157s 6s/step - loss: 1.2361 - accuracy: 0.6339 - val_loss: 0.4264 - val_accuracy: 0.8021
Epoch 3/20
25/25 [==============================] - 166s 7s/step - loss: 0.7336 - accuracy: 0.6921 - val_loss: 0.4534 - val_accuracy: 0.8542
Epoch 4/20
25/25 [==============================] - 167s 7s/step - loss: 0.5979 - accuracy: 0.7180 - val_loss: 0.4078 - val_accuracy: 0.8490
Epoch 5/20
25/25 [==============================] - 170s 7s/step - loss: 0.5159 - accuracy: 0.7490 - val_loss: 0.3631 - val_accuracy: 0.8594
Epoch 6/20
25/25 [==============================] - 195s 8s/step - loss: 0.4470 - accuracy: 0.7930 - val_loss: 0.3084 - val_accuracy: 0.8698
Epoch 7/20
25/25 [==============================] - 189s 8s/step - loss: 0.4146 - accuracy: 0.7930 - val_loss: 0.3913 - val_accuracy: 0.8333
Epoch 8/20


In [20]:
# Evaluate the model
accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy[1]*100:.2f}%")

7/7 [==============================] - 25s 3s/step - loss: 0.2090 - accuracy: 0.9356
Test Accuracy: 93.56%


In [21]:
# Save the trained model with a different name
model.save('New_cataract_model.h5')

c:\conda\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
# Alternatively, you can save it in the same folder as the dataset
model.save(os.path.join(dataset_dir, 'New_cataract_model.h5'))